# Code for estimating observed counts from Landolt

### Loads in data for the orbit and location of Landolt
satcoord.csv -> data from Landolt orbit simulations\
satlatlon.csv -> latitude and longitude of Landolt\
satcoordxyz.csv -> cartesian coordinates of Landolt

In [1]:
import numpy as np
from scipy.integrate import quad
from settings import parameters
import sys
import matplotlib.pyplot as plt

data = np.genfromtxt('satcoord.csv',delimiter=',',skip_header=1)
datalatlon = np.genfromtxt('satlatlon.csv',delimiter=',',skip_header=1)
dataxyz = np.genfromtxt('satcoordxyz.csv',delimiter=',',skip_header=1)

### Assigns parameters from settings.json and above files to variables
z -> distance from observer to Landolt (m)\
alt -> altitude of Landolt at the center of the beam path (rads)\
alt0 -> altitude of Landolt at the center of the beam path when observations begin (rads)\
tdelta -> time increment of satcoord.csv orbit simulation calculations (s)\
lat_obs, lon_obs -> latitude and longitude of the center of the beam path (rads)\
lat_loc, lon_loc -> latitude and longitude of observer (rads)\
t_efficiency -> telescope efficiency\
ccd_efficiency -> ccd quantum efficiency\
diam_t -> diameter of observer's telescope (m)\
a_t -> area over which the telescope takes in light (m^2)\
lmbda_n -> parameter that determines which laser is being observed\
(0 - 355nm, 1 - 488nm, 2 - 655nm, 3 - 785nm, 4 - 976nm, 5 - 1064nm, 6 - 1310nm, 7 - 1550nm)\
humidity -> relative humidity at observation location (%)\
aod -> aerosol optical depth at each wavelength\
alpha -> angle that a line perpendicular to the center of the beam path makes with a line tangent to Earth's surface at the center of the beam path (rads)\
t -> array of times incrementing with t=tdelta (s)\
airmass -> airmass at the altitude of Landolt

In [2]:
z = data[:,5]*1e3
alt = data[:,4]*(np.pi/180) 
alt0 = data[0,4]*(np.pi/180) 
tdelta = parameters.tdelta/1000
lat_obs = parameters.lat*(np.pi/180)
lon_obs = parameters.lon*(np.pi/180)
lat_loc = float(parameters.lat_loc)*(np.pi/180)
lon_loc = float(parameters.lon_loc)*(np.pi/180) 
t_efficiency = float(parameters.t_eff)
ccd_efficiency = float(parameters.ccd_eff)
diam_t = float(parameters.t_diam)
a_t = np.pi*(diam_t/2)**2
lmbda_n = int(parameters.n)
humidity = float(parameters.humidity)
fob = 1
alpha = np.pi/2 - alt
t = np.linspace(0,len(z)-1,num=len(z))*tdelta
airmass = (1/np.cos(alpha)) - 0.0018167*((1/np.cos(alpha))-1) - 0.002875*((1/np.cos(alpha))-1)**2 - 0.0008083*((1/np.cos(alpha))-1)**3
aod = [0.055, 0.08, 0.06, 0.045, 0.045, 0.045, 0.035, 0.035]
# aod varies w/ humidity, the code factors that in here
if humidity >= 0.6:
    aod[lmbda_n] = aod[lmbda_n] + 0.05
if humidity >= 0.8:
    aod[lmbda_n] = aod[lmbda_n] + 0.05

### Creating Empty Variable Tables 

In [3]:
w_z = np.zeros(len(z))
FWHM = np.zeros(len(z))
error_p = np.zeros(len(z))
z_new = np.zeros(len(t))
w_z = np.zeros(len(t))
flux_z = np.zeros(len(t))
tflux = np.zeros(len(t))
counts = np.zeros(len(t))
I_final = np.zeros(len(t))
counts_final = np.zeros(len(t))
mag_final = np.zeros(len(t))
num_flux = np.zeros(len(t))
num_counts = np.zeros(len(t))
num_I_final = np.zeros(len(t))
num_counts_final = np.zeros(len(t))
t_reqd = np.zeros(len(t))
num_t_reqd = np.zeros(len(t))
curve_theta = np.zeros(len(t))

### Finding the position of observer relative to Landolt
orient_x, orient_y, orient_z -> x, y, and z coordinates of the center of the beam path evaluated from the latitude and longitude of Landolt using spherical coordinates and using the volumetric mean radius of Earth (m)\
orient_xloc, orient_yloc, orient_zloc -> x, y, and z coordinates of the observer evaluated similarly with the center of the beam path as the origin (m)\
sat_x, sat_y, sat_z -> x, y, and z coordinates of Landolt in GCRS coordinates (m)\
sat_lat, sat_lon -> latitude and longitude of Landolt projected to Earth (rads)\
orient_xsat, orient_ysat, orient_zsat -> x, y, and z coordinates of Landolt projected on to Earth's surface with the center of the beam path as the origin evaluated from its latitude and longitude using spherical coordinates (m)\
d0 -> distance of the observer from the center of the beam path evaluated from the x, y, and z coordinates calculated above (m)\
beta -> the angle between the orient_x/y/zloc and orient_x/y/zsat vectors calculated from the equation for the dot product of two vectors (rads)

In [4]:
orient_x = 6371000*np.sin((np.pi/2) - lat_obs)*np.cos(lon_obs)
orient_y = 6371000*np.sin((np.pi/2) - lat_obs)*np.sin(lon_obs)
orient_z = 6371000*np.cos((np.pi/2) - lat_obs)
orient_xloc = 6371000*np.sin((np.pi/2) - lat_loc)*np.cos(lon_loc)
orient_yloc = 6371000*np.sin((np.pi/2) - lat_loc)*np.sin(lon_loc)
orient_zloc = 6371000*np.cos((np.pi/2) - lat_loc)
orient_xloc = orient_xloc - orient_x # to get vector from center of the beam path to observer
orient_yloc = orient_yloc - orient_y # similarly
orient_zloc = orient_zloc - orient_z # similarly
sat_x = dataxyz[:,0]
sat_y = dataxyz[:,1]
sat_z = dataxyz[:,2]
sat_lat = datalatlon[:,0]
sat_lon = datalatlon[:,1]
orient_xsat = 6371000*np.sin((np.pi/2) - sat_lat)*np.cos(sat_lon)
orient_ysat = 6371000*np.sin((np.pi/2) - sat_lat)*np.sin(sat_lon)
orient_zsat = 6371000*np.cos((np.pi/2) - sat_lat)
orient_xsat = orient_xsat - orient_x # to get vector from center of the beam path to observer
orient_ysat = orient_ysat - orient_y # similarly
orient_zsat = orient_zsat - orient_z # similarly
d0 = np.sqrt(orient_xloc**2 + orient_yloc**2 + orient_zloc**2) # distance of observer from center of beam path
beta = np.arccos(((orient_xloc*orient_xsat) + (orient_yloc*orient_ysat) + (orient_zloc*orient_zsat))/(d0*np.sqrt(orient_xsat**2 + orient_ysat**2 + orient_zsat**2))) # angle between a line made between the center of the beam path and observer and the satellite-to-center of beam path vector projected on to earth's surface
if d0 < diam_t/2:
    d0 = diam_t/2 # fixes error where starting at zero creates invalid variables, sets distance from center of the beam path to the radius of the telescope at the very minimum


### Laser properties and magnitude zero points
MFD -> Mode field diameter for each laser (m)\
w_0 -> Initial waist radius of the Gaussian beam (m)\
lmbda -> Wavelengths of each laser (m)\
P_0 -> Total output power of each laser (W)\
zp -> Zero points for the wavelengths of each laser

In [7]:
MFD = [2.5e-6, 3.5e-6, 4e-6, 5e-6, 5.9e-6, 6.2e-6, 9.2e-6, 10.4e-6] # mode field diameter of optical fiber
w_0 = MFD[lmbda_n]/2 # waist radius of the gaussian beam
lmbda = [355e-9, 488e-9, 655e-9, 785e-9, 976e-9, 1064e-9, 1310e-9, 1550e-9] # wavelength of all eight lasers
P_0 = [0.003, 0.04, 0.05, 0.0636, 0.45, 0.3, 0.5, 0.1] # power of all four lasers
zp = [417.5*1e-7*10000*a_t*1e10*lmbda[0]*1e-11, 632*1e-7*10000*a_t*1e10*lmbda[1]*1e-11, 217.7*1e-7*10000*a_t*1e10*lmbda[2]*1e-11, 112.6*1e-7*10000*a_t*1e10*lmbda[3]*1e-11, 31.47*1e-7*10000*a_t*1e10*lmbda[4]*1e-11, 31.47*1e-7*10000*a_t*1e10*lmbda[5]*1e-11, 31.47*1e-7*10000*a_t*1e10*lmbda[6]*1e-11, 11.38*1e-7*10000*a_t*1e10*lmbda[7]*1e-11] # zero points of each laser

### Preliminary calculations
I_0 -> Incident intensity of the specified laser (W/m^2)\
z_r -> Raleigh range used in calculating the radius of the Gaussian beam at a given distance z (https://www.thorlabs.com/newgrouppage9.cfm?objectgroup_id=14204) (m)\
w_z0 -> Gaussian beam radius at a distance z (https://www.thorlabs.com/newgrouppage9.cfm?objectgroup_id=14204) (m)\
FWHM -> full width at half maximum of a Gaussian beam profile with a radius of w_z0\
x -> The array of distances to be used in numerically integrating the beam profile over the diameter of the observer's telescope (m)\
theta -> The angle made between the normal vector of Earth's surface at the center of the beam path and a ray of light landing a given distance away from the normal vector. This is used in determining how different parts of the Gaussian beam reach Earth at different distances and various angles of incidence (rads)

In [8]:
I_0 = (2*P_0[lmbda_n])/(np.pi*w_0**2) # incident intensity of the laser
z_r = (np.pi/lmbda[lmbda_n])*w_0**2 # raleigh range

# calculates flux as a gaussian distribution for height above center of beam path given
w_z0 = w_0*np.sqrt(1+(z[0]/z_r)**2) # beam radius at distance z
if d0 > w_z0:
    print('Error: Observer Outside Beam Path')
    sys.exit()
FWHM = np.sqrt(2*np.log(2))*w_z0 # full width at half maximum of the beam profile for a given distance from the waist
x = np.arange(d0 - diam_t/2, d0 + diam_t/2, 0.001) # the distance on one direction perpendicular to the laser vector
theta = np.arctan(d0/z) # angle made between the normal of earth's surface and a beam of light landing a given distance away from the normal

### Analytic Gaussian distribution of flux
The distances z from Landolt to the surface of Earth are first modified by accounting for Earth's curvature. A cos^-1(theta) term is also applied to the original z values to take into account the incidence angle of the Gaussian beam. To account for the projection of the beam on a tangent line at the center of the beam path, trigonometry is used to determine to which extent the distances z change at different positions in the beam's projection on Earth's surface. For instance, at an altitude of 45 degrees, z distances increase towards Landolt and away from Landolt but remain constant perpendicular to the beam's path to Earth. This code also accounts for how the changes in altitude effect how much these distances increase and decrease.

In [9]:
print('Calculating Gaussian distribution of flux...')
for j in range(len(t)):
    curve_theta[j] = np.arctan(d0/6371000) # angle between the center of the beam path and the observer measured from the center of the earth
    z_new[j] = (z[j]+(6371000-6371000*np.cos(curve_theta[j])))/np.cos(theta[j]) # amount of distance a given light ray travels factoring in the curvature of the earth
    if alt0 <= alt[j]: # identifies if observer is closer or further from the satellite using its relative altitude in the sky
        z_new[j] = z_new[j] - d0*np.tan(alpha[j])*np.sin(beta[j])
    else:
        z_new[j] = z_new[j] + d0*np.tan(alpha[j])*np.sin(beta[j])
    w_z[j] = w_0*np.sqrt(1+(z_new[j]/z_r)**2) # beam radius observed on earth's surface accounting for the curvature of earth
    flux_z[j] = I_0*((w_0/w_z[j])**2)*np.e**((-2*d0**2)/w_z[j]**2) # flux along one 2D slice of the 3D gaussian beam profile for different distances from the satellite in the center of the beam path
print('Done!')

Calculating Gaussian distribution of flux...
Done!


### Calculating flux reeived at telescope analytically
error_p -> error in pointing (m)\
The total flux is first found by first integrating the 2D Gaussian distribution of flux across a segment the length of the diameter of the telescope centered around the distance of the observer from the center of the beam path. To take into account the 3D nature of the beam, the area a ring with inner radius d0 - diam_t/2 and outer radius d0 + diam_t is first divided by the area the telescope is able to take in light. The 2D flux is then multiplied by 2pi divided by this value to integrate across phi in spherical coordinates. Integration is able to be done this way because the Gaussian beam is radially symmetric.\
tflux -> power received by telescope ignoring atmospheric extinction (W)\
counts -> counts received by telescope ignoring atmospheric extinction (photons/s)

In [74]:
dis_t = x # renaming variable
error_p[1:] = z_new[1:]*0.00000484813681109536*t[1:]

print('Calculating flux recieved at telescope...')
for i in range(len(t)):
    def flux_fn(r):
        return r*I_0*((w_0/w_z[i])**2)*np.e**((-2*r**2)/w_z[i]**2)
    tflux_temp = quad(flux_fn, -(diam_t/2) + d0, (diam_t/2) + d0)
    coeftemp = np.pi*(((diam_t/2) + d0)**2 - (-(diam_t/2) + d0)**2) / a_t
    tflux[i] = tflux_temp[0]*(2*np.pi/coeftemp)
    counts[i] = (tflux[i]*lmbda[lmbda_n])/(6.62607015e-34*299792458)
print('Done!')


Calculating flux recieved at telescope...
Done!


### Calculating the flux received at telescope numerically
The same procedure as above is used, though this time a Riemann sum is taken of any given distribution of flux. Each component of the Riemann sum is subject to the same 3D integration detailed above.\
num_flux -> numerically calculated power received by telescope ignoring atmospheric extinction (W)\
num_counts -> numerically calculated counts received by telescope ignoring atmospheric extinction (photons/s)

In [75]:
print('Calculating flux received at telescope numerically... (this will take a while)')
for i in range(len(t)):
    num_flux_temp = 0
    r_sum = 0
    for j in range(len(dis_t) - 1):
        def flux_fn(r): # defines the function for the distribution of light -> can be replaced with anything
            return I_0*((w_0/w_z[i])**2)*np.e**((-2*r**2)/w_z[i]**2)    
        r_sum = flux_fn((dis_t[j+1] + dis_t[j])/2)*(dis_t[j+1] - dis_t[j])
        num_flux_temp = num_flux_temp + r_sum
    coeftemp = np.pi*(((diam_t/2) + (dis_t[0] + dis_t[len(dis_t)-1])/2)**2 - (-(diam_t/2) + (dis_t[0] + dis_t[len(dis_t)-1])/2)**2) / a_t
    num_flux[i] = d0*num_flux_temp*(2*np.pi/coeftemp)
    num_counts[i] = (num_flux[i]*lmbda[lmbda_n])/(6.62607015e-34*299792458)
print('Done!')

Calculating flux received at telescope numerically... (this will take a while)
Done!


### Factoring in atmospheric extinction
r_coef -> Atmospheric extinction coefficient found using the Beer-Lambert law where tau is the Rayleigh cross section, distance from Landolt to observer, and average number of molecules per cubic meter multiplied together. r_coef is calculated for n2, o2, argon, co2, and neon\
m_coef -> transmission coefficient from mie scattering\
N -> Average number of molecules per square meter in Earth's atmosphere. This is calculated assuming all of Earth's atmosphere lies between Landolt and Earth's surface\
cs_n2, cs_o2, cs_ar, cs_co2, cs_ne -> rayleigh scattering cross sections for n2, o2, argon, co2, and neon\
I_final -> calculates power observed at telescope factoring in atmospheric extinction (W)\
counts_final -> calculates counts observed at telescope factoring in atmospheric extinction (photons/s)\
mag_final -> calculates magnitude observed at telescope factoring in atmospheric extinction\
num_I_final, num_counts_final -> the above values calculated numerically\
t_reqd -> estimated exposure time necessary to receive 4.4e5 counts (s)\
num_t_reqd -> estimated exposure time necessary to receive 4.4e5 counts calculated numerically (s)

In [77]:
def r_coef(cs, d, N):
    """
    Using the Rayleigh scattering cross section cs, the 
    scattering coefficient is found given the distance d from the satellite to the detector,
    and the amount of molecules N per cubic meter.
    """
    r_coef = np.e**(-cs*N*d)
    return r_coef

N = 1e44 / (((4/3)*np.pi*(6371000 + z_new)**3) - (4/3)*np.pi*(6371000)**3)# average number of molecules that measured light passes per square meter

# rayleigh scattering cross sections for 355 nm for the five most abundant gases in the atmosphere
if lmbda_n == 0:
    cs_n2 = 23.82e-31
    cs_o2 = 20.03e-31
    cs_ar = 23e-31
    cs_co2 = 70.70e-31
    cs_ne = 1.01e-31

# rayleigh scattering cross sections for 488 nm for the five most abundant gases in the atmosphere
elif lmbda_n == 1:
    cs_n2 = 7.26e-31
    cs_o2 = 6.50e-31
    cs_ar = 7.24e-31
    cs_co2 = 23e-31
    cs_ne = 0.33e-31

# rayleigh scattering cross sections for 655 nm for the five most abundant gases in the atmosphere
# these values are extrapolated assuming a direct 1/lambda^4 relationship
elif lmbda_n == 2:
    cs_n2 = 2.24e-31
    cs_o2 = 2.06e-31
    cs_ar = 2.08e-31
    cs_co2 = 7.28e-31
    cs_ne = 0.103e-31

# rayleigh scattering cross sections for 785 nm for the five most abundant gases in the atmosphere
# these values are extrapolated assuming a direct 1/lambda^4 relationship
elif lmbda_n == 3:
    cs_n2 = 2.65e-31
    cs_o2 = 2.2e-31
    cs_ar = 2.38e-31
    cs_co2 = 6.22e-31
    cs_ne = 0.128e-31

# rayleigh scattering cross sections for 976 nm for the five most abundant gases in the atmosphere
# these values are extrapolated assuming a direct 1/lambda^4 relationship
elif lmbda_n == 4:
    cs_n2 = 1.11e-31
    cs_o2 = 0.92e-31
    cs_ar = 0.97e-31
    cs_co2 = 2.6e-31
    cs_ne = 0.128e-31

# rayleigh scattering cross sections for 1064 nm for the five most abundant gases in the atmosphere
# these values are extrapolated assuming a direct 1/lambda^4 relationship
elif lmbda_n == 5:
    cs_n2 = 0.79e-31
    cs_o2 = 0.65e-31
    cs_ar = 0.68e-31
    cs_co2 = 1.84e-31
    cs_ne = 3.79e-33

# rayleigh scattering cross sections for 1310 nm for the five most abundant gases in the atmosphere
# these values are extrapolated assuming a direct 1/lambda^4 relationship
elif lmbda_n == 6:
    cs_n2 = 0.34e-31
    cs_o2 = 0.28e-31
    cs_ar = 0.30e-31
    cs_co2 = 0.8e-31
    cs_ne = 1.65e-33

# rayleigh scattering cross sections for 1550 nm for the five most abundant gases in the atmosphere
# these values are extrapolated assuming a direct 1/lambda^4 relationship
else:
    cs_n2 = 7.13e-33
    cs_o2 = 6.39e-33
    cs_ar = 7.11e-33
    cs_co2 = 2.26e-32
    cs_ne = 3.24e-34

r_coef1 = 1 - r_coef(cs_n2, z_new, N*0.78084) # scattering coefficient from rayleigh scattering for n2
r_coef2 = 1 - r_coef(cs_o2, z_new, N*0.20946) # scattering coefficient from rayleigh scattering for o2
r_coef3 = 1 - r_coef(cs_ar, z_new, N*0.00934) # scattering coefficient from rayleigh scattering for argon
r_coef4 = 1 - r_coef(cs_co2, z_new, N*0.000397) # scattering coefficient from rayleigh scattering for co2
r_coef5 = 1 - r_coef(cs_ne, z_new, N*1.818e-5) # scattering coefficient from rayleigh scattering for neon
m_coef = 1 - np.ones(len(t))*np.e**(-aod[lmbda_n]) # transmission coefficient from mie scattering

for i in range(len(t)):
    I_final[i] = (tflux[i] - tflux[i]*(m_coef[i]+r_coef1[i]+r_coef2[i]+r_coef3[i]+r_coef4[i]+r_coef5[i])*airmass[i])*t_efficiency # calculates flux observed at telescope
    counts_final[i] = ((I_final[i]*lmbda[lmbda_n])/(6.62607015e-34*299792458))*ccd_efficiency # total counts taken in
    mag_final[i] = -2.5*np.log10(I_final[i]/zp[lmbda_n]) # relative magnitude calculated from vega zero points
    num_I_final[i] = (num_flux[i] - num_flux[i]*(m_coef[i]+r_coef1[i]+r_coef2[i]+r_coef3[i]+r_coef4[i]+r_coef5[i])*airmass[i])*t_efficiency # calculates numerical flux observed at telescope
    num_counts_final[i] = ((num_I_final[i]*lmbda[lmbda_n])/(6.62607015e-34*299792458))*ccd_efficiency # conversion from numerically calculated flux to photoelectric counts
    t_reqd[i] = 4.4e5 / counts_final[i] # amount of seconds needed to observe 4.4e5 counts
    num_t_reqd[i] = 4.4e5 / num_counts_final[i] # same as above but for the numerical counts

### Formatting satcoord.csv for export
This section incorperates the blinking of the laser and adds columns for radiant flux, counts per second, airmass, magnitude, and recommended exposure time to the satcoord.csv file

In [ ]:
for i in range(int(fob/1e-3)):
    I_final[i::2*int(fob/1e-3)] = 0
    counts_final[i::2*int(fob/1e-3)] = 0
    mag_final[i::2*int(fob/1e-3)] = 0
    num_I_final[i::2*int(fob/1e-3)] = 0
    num_counts_final[i::2*int(fob/1e-3)] = 0
    
I_final = I_final[0:len(t)]
counts_final = counts_final[0:len(t)]
mag_final = mag_final[0:len(t)]

print('Exporting file...')
heading = np.array('Radiant Flux (W)',dtype='str')
heading2 = np.array('Counts per Second')
heading3 = np.array('Airmass')
heading4 = np.array('Magnitude')
heading5 = np.array('Recommended Exposure Time (s)')
data = np.genfromtxt('satcoord.csv',dtype='str',delimiter=',')
data = data[:,:7]
I_final = np.asarray(I_final,dtype='str')
counts_final = np.asarray(counts_final,dtype='str')
airmass = np.asarray(airmass,dtype='str')
mag_final = np.asarray(mag_final,dtype='str')
t_reqd = np.asarray(t_reqd,dtype='str')
I_final = np.insert(I_final[:],0,heading)
counts_final = np.insert(counts_final[:],0,heading2)
airmass = np.insert(airmass[:],0,heading3)
mag_final = np.insert(mag_final[:],0,heading4)
t_reqd = np.insert(t_reqd[:],0,heading5)
output = np.column_stack((data,I_final))
output = np.column_stack((output,counts_final))
output = np.column_stack((output,airmass))
output = np.column_stack((output,mag_final))
output = np.column_stack((output,t_reqd))
np.savetxt('satcoord.csv', output, fmt='%s', delimiter=',')

print('Done!')